In [1]:
import os
from PIL import Image
from glob import glob
import numpy as np
import tensorflow as tf
!pip install pydicom
import pydicom
from IPython import display
!pip install xlrd
import xlrd
# check generated positive samples
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import tensorflow as tf
from copy import copy
import matplotlib.colors as colors

--AA
AA--
None
--AA
AA--


In [2]:


def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# Create a dictionary with features that may be relevant. # create tf.example message
def image_example(each_dicom_path, seg_label, cls_label):
    feature = {
      
        'dicom_path': _bytes_feature(each_dicom_path.encode()),
        'seg_label': _bytes_feature(seg_label.tostring()), 
        'cls_label': _int64_feature(cls_label),
        
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))


def image_pair_example(each_dicom_path1, each_dicom_path2, each_dicom_path3, seg_label1, seg_label2, seg_label3, cls_label1, cls_label2,  cls_label3):
    feature = {
      
        'dicom_path1': _bytes_feature(each_dicom_path1.encode()),
        'seg_label1': _bytes_feature(seg_label1.tostring()), 
        'cls_label1': _int64_feature(cls_label1),
        
        'dicom_path2': _bytes_feature(each_dicom_path2.encode()),
        'seg_label2': _bytes_feature(seg_label2.tostring()), 
        'cls_label2': _int64_feature(cls_label2),
        
        'dicom_path3': _bytes_feature(each_dicom_path3.encode()),
        'seg_label3': _bytes_feature(seg_label3.tostring()), 
        'cls_label3': _int64_feature(cls_label3),
        
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

def make_sample_pairs(root_folder_path_for_cases, excel_loc, train_or_val, marked_datatype= np.ubyte):
    # get case_list
    case_list = glob(root_folder_path_for_cases + "*")
    # print("case_list:", case_list)
    print("len of case_list:", len(case_list))
    
    
    # get work_book of excel
    train_wb = xlrd.open_workbook(excel_loc)
    # get train filp colomn
    sheet =  train_wb.sheet_by_index(0)
    print("sheet:", sheet)

    # extract_number_of rows
    print(sheet.nrows)
    
    case_names =  []
    case_flips = []
    for i in range(1, sheet.nrows):
        case_names.append(sheet.cell_value(i, 0))
        case_flips.append(sheet.cell_value(i, 1))

    
    # convert flip notes into integer 
    print("case_flips:", case_flips)
    case_flips =[int(s) for s in case_flips] 
    print("case_names:", case_names)
    print("case_flips:", case_flips)
    print("len of case data:", len(case_names))
    print("len of flip data:", len(case_flips))

    # check case list
    print("case list len:", len(case_list))
    assert len(case_names) ==  len(case_flips), " len of cases names does not macthed with flip len in train excel "
    assert len(case_list) ==  len(case_names), " len of cases in the train folder not macthed with cases in train excel "
    
    # one feature contains : input 1 , seg1, cls1, input 2, seg2, cls2, input3, seg3, cls3 with shape [1, 9]
    
    positive_sample_features =  []
    negative_sample_features =  []
    
    
    # loop each case folder to select samples
    for case_index, each_case in enumerate(case_list):
        display.clear_output(wait=True)
        print("case index:{}, case:{}".format(case_index, each_case))
        each_case_str = os.path.split(each_case) 
        print("each_case_str:", each_case_str)
        case_name =  each_case_str[-1]
        print("case name:", case_name)
        print("case index:", case_index, "case name:", case_names[case_index])
        # check case name match or not
        assert case_name ==  case_names[case_index]
        
        # get dicom files and label file
        # read_labels for specified case name
        # read rawfile to get seg labels for the entire case: ----------------------->  for seg label 
        raw_files = glob(each_case + "/*.raw")
        # get label path
        for each_path in raw_files:
            if "marked" in each_path:
                label_path =  each_path
        print("label path:", label_path)
        # read labels
        labels = np.fromfile(label_path, dtype = marked_datatype)
        reshaped_labels =  labels.reshape([-1, 512, 512])
        print("reshape labels shape:", reshaped_labels.shape)
#

       # check whether needs to flip the label
        if case_flips[case_index] ==1: # flip the enitre label
            final_case_labels = np.flip(reshaped_labels, 0)
        else: 
            final_case_labels =  reshaped_labels
        
        
        step=3 # number of elements for each sample
        
        # rearrange the label array ever 3 elemtns step 1
        rearrange_3_seg_labels_list = []
        
       
        for i in range(0, final_case_labels.shape[0]-step+1):
            rearrange_3_seg_labels_list.append(final_case_labels[i:i+step])
        print("rearrange_3labels_list len---->:", len(rearrange_3_seg_labels_list))
        final_seg_label_list = rearrange_3_seg_labels_list
        
        
        
        
        # get dicom list...-------------------------------->   for dicom input path
        dicom_files = glob(each_case + "/*.DCM")
        
        # in order for ubuntu system, needs to rewrite the path of dicom
        dicom_path_list =[]
        cls_list=[]
        for dicom_index, each_dicom in enumerate(dicom_files):
            path_split = os.path.split(each_dicom)
#             print(path_split)
            file_name =  path_split[-1]
#             print(file_name)


            each_slice_label =  final_case_labels[dicom_index]
            # count non_zeros value from the each slice label
            num_nonzero = np.count_nonzero(each_slice_label)
            
            
#             # for lab ubuntu
#             if train_or_val == "train": # change replace paths considering for ubuntu path root changed
#                 each_dicom =  "/media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/" + case_name +"/" +file_name
#             else:
#                 each_dicom =  "/media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/val/" + case_name+ "/" +file_name
            # for lab windows
            if train_or_val == "train": # change replace paths considering for ubuntu path root changed
                each_dicom =  "F:\\dataset\\Leisang\\myTry\\BleedingDataDCM\\train/" + case_name +"/" +file_name
            else:
                each_dicom =  "F:\\dataset\\Leisang\\myTry\\BleedingDataDCM\\val/" + case_name+ "/" +file_name
            
            
            if num_nonzero > 10:  # only consider the # of nonzeros values >10 to be the positive labels
                each_slice_cls_label = 1
                print("num_nonzero:", num_nonzero)
#                 None_zero_label_index.append(dicom_index+1)
                print("each_slice_label shape:", each_slice_label.shape)
                print("each_slice label range before binarization: [{}, {}]".format(each_slice_label.min(), each_slice_label.max()))

                # binarize the mask label
                each_slice_label[each_slice_label> 0] = 255
                print("each_slice label range before after binarization: [{}, {}]".format(each_slice_label.min(), each_slice_label.max()))
            else: # negative examples
                each_slice_cls_label = 0
            
            if dicom_index % 20 == 0:
                    display.clear_output(wait=True)
            
            dicom_path_list.append(each_dicom)
            cls_list.append(each_slice_cls_label)
        
        # rearrange dicom path list and cls_list into 3 elements per group
        final_dicom_path_list = [dicom_path_list[i:i+step] for i in range(0, len(dicom_path_list)-step+1)]
        final_cls_list = [cls_list[i:i+step] for i in range(0, len(cls_list)-step+1)]
        
        print("final_dicom_path_list:", len(final_dicom_path_list))
#         print(final_dicom_path_list)
        print("final_cls_list:", len(final_cls_list))
#         print(final_cls_list)
        
#         # reform the dicom list into 3 elements per step
      
#         dicom_3_files_list = [dicom_files[i:i+step] for i in range(0, len(dicom_files)-step+1)]
#         print("dicom_3_files_list len---->:", len(dicom_3_files_list))
        print("dicom_files[0]:", dicom_files[0])
        print("final_dicom_path_list[0][0]:", final_dicom_path_list[0][0])
        print("dicom_files[-1]:", dicom_files[-1])
        print("final_dicom_path_list[-1][-1]:", final_dicom_path_list[-1][-1])
        print("final_dicom_path_list[0]:", final_dicom_path_list[0])
        print("final_dicom_path_list[1]:", final_dicom_path_list[1])
        
        print()
        print("final case label shape:", final_case_labels.shape)
#         assert os.path.normpath(dicom_files[0]) == os.path.normpath(final_dicom_path_list[0][0])  # Use os.path.normpath to convert c:/fold1/fold2 to c:\fold1\fold2
#         assert os.path.normpath (dicom_files[-1]) == os.path.normpath(final_dicom_path_list[-1][-1])
        print("len of dicoms in this case:", len(dicom_files))
        assert  reshaped_labels.shape[0] ==  len(dicom_files), 'len of dicoms is not equal to the raw label values'
        print()    
        
        # pack the data with as [(inputs1, seg_tar1, cls_tar1) , （inputs2, seg_tar2, cls_tar2）， (inputs3, seg_tar3, cls_tar3)] 
        final_samples =  zip(final_dicom_path_list, final_seg_label_list, final_cls_list)
#         print("len of case samples:", list(final_samples))
        final_case_samples_list =  list(final_samples)
#         print("final_case_samples_list[0]:",  final_case_samples_list[0]) 
        # Consider add_positive samples and negative samples
        for i in range(len(final_cls_list)):
            if i  % 20 == 0:
                display.clear_output(wait=True)
            print("i:",i)
            print("final_cls_list[i]:", final_cls_list[i])
#             print("final_clas_list[i]:", final_cls_list[i])
            num_non_zero_clses = np.count_nonzero(final_cls_list[i])
            print("num_non_zero_clses", num_non_zero_clses)
            if num_non_zero_clses >1:
#                 print("final_case_samples_list[i]:", final_case_samples_list[i])
                positive_sample_features.append(final_case_samples_list[i])
            else: 
#                 print("final_case_samples_list[i]:", final_case_samples_list[i])
                negative_sample_features.append(final_case_samples_list[i])
            
           
                
    print("total positive samples:",len(positive_sample_features))
    print("total negative samples:",len(negative_sample_features))            
    return positive_sample_features, negative_sample_features
        
# def make_samples_win(root_folder_path_for_cases, excel_loc, train_or_val, marked_datatype= np.ubyte):
    
    
#     # get case_list
#     case_list = glob(root_folder_path_for_cases + "*")
#     # print("case_list:", case_list)
#     print("len of case_list:", len(case_list))
    
    
#     # get work_book of excel
#     train_wb = xlrd.open_workbook(excel_loc)
#     # get train filp colomn
#     sheet =  train_wb.sheet_by_index(0)
#     print("sheet:", sheet)

#     # extract_number_of rows
#     print(sheet.nrows)
    
#     case_names =  []
#     case_flips = []
#     for i in range(1, sheet.nrows):
#         case_names.append(sheet.cell_value(i, 0))
#         case_flips.append(sheet.cell_value(i, 1))

#     # convert flip notes into integer 
#     print("case_flips:", case_flips)
#     case_flips =[int(s) for s in case_flips] 
#     print("case_names:", case_names)
#     print("case_flips:", case_flips)
#     print("len of case data:", len(case_names))
#     print("len of flip data:", len(case_flips))

#     # check case list
#     print("case list len:", len(case_list))
#     assert len(case_names) ==  len(case_flips), " len of cases names does not macthed with flip len in train excel "
#     assert len(case_list) ==  len(case_names), " len of cases in the train folder not macthed with cases in train excel "

#     my_positive_exmaple ={}
#     my_negative_exmaple={}
    
#     for case_index, each_case in enumerate(case_list):
#         print(each_case)
#         # get case name from case list:
#         each_case_str = os.path.split(each_case) 
#         print("each_case_str:", each_case_str)
#         case_name =  each_case_str[-1]
#         print("case name:", case_name)
#         print("case index:", case_index, "case name:", case_names[case_index])
#         # check case name match or not
#         assert case_name ==  case_names[case_index]

#         # read_labels for specified case name
#         raw_files = glob(each_case + "/*.raw")
#         dicom_files = glob(each_case + "/*.DCM")
        
        
#         # get label path
#         for each_path in raw_files:
#             if "marked" in each_path:
#                 label_path =  each_path
#         print("label path:", label_path)
#         # read labels
#         labels = np.fromfile(label_path, dtype = marked_datatype)
#         reshaped_labels =  labels.reshape([-1, 512, 512])
#         print("reshape labels shape:", reshaped_labels.shape)
#         print("len of dicoms in this case:", len(dicom_files))
#         assert  reshaped_labels.shape[0] ==  len(dicom_files)  , 'len of dicoms is not equal to the raw label values'

#        # check whether needs to flip the label
#         if case_flips[case_index] ==1: # flip the enitre label
#             train_final_case_labels = np.flip(reshaped_labels, 0)
#         else: 
#             train_final_case_labels =  reshaped_labels

#         None_zero_label_index = []
#         # read inputs
#         for dicom_index, each_dicom in enumerate(dicom_files):
#             path_split = os.path.split(each_dicom)
#             print(path_split)
#             file_name =  path_split[-1]
#             print(file_name)
            
           
#             each_slice_label =  train_final_case_labels[dicom_index]
#             # count non_zeros value from the each slice label
#             num_nonzero = np.count_nonzero(each_slice_label)
            
#             if train_or_val == "train": # change replace paths considering for ubuntu path root changed
#                 each_dicom =  "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM\\train/" + case_name +"/" +file_name
#             else:
#                 each_dicom =  "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM\\val/" + case_name+ "/" +file_name
            
#             if num_nonzero > 10:  # only consider the # of nonzeros values >10 to be the positive labels
#                 each_slice_cls_label = 1
#                 print("num_nonzero:", num_nonzero)
#                 None_zero_label_index.append(dicom_index+1)
#                 print("each_slice_label shape:", each_slice_label.shape)
#                 print("each_slice label range before binarization: [{}, {}]".format(each_slice_label.min(), each_slice_label.max()))

#                 # binarize the mask label
#                 each_slice_label[each_slice_label> 0] = 255
#                 print("each_slice label range before after binarization: [{}, {}]".format(each_slice_label.min(), each_slice_label.max()))

#                 # genrate positive examples dict = {"path"： label}
#                 my_positive_exmaple[each_dicom]=(each_slice_label, each_slice_cls_label)

#             else: # negative examples
#                 each_slice_cls_label = 0
#                 my_negative_exmaple[each_dicom]=(each_slice_label, each_slice_cls_label)

#             if dicom_index % 20 == 0:
#                     display.clear_output(wait=True)  

#             if dicom_index == len(dicom_files)-1:
#                 print("total " + str(len(dicom_files)) + " checked ---------------------------------------------------------------------------->")

   
#         # check the length for each case
#         print("generate positive {} positive samples at currently case step:".format(len(my_positive_exmaple)))
#         print("generate positive {} negative samples at currently case step:".format(len(my_negative_exmaple)))   
#     return my_positive_exmaple, my_negative_exmaple


def Make_InputPairsTFrecords(sample_list, file_nameToSave):
    from pathlib import Path
    # start to write 
    print("writing tfrecords....")
   
    # strat to write postive train dicoms
    with tf.io.TFRecordWriter(file_nameToSave) as writer:
        print("paths sample_list[0][0]:", sample_list[0][0])
        print("seg_tars sample_list[0][1]:", sample_list[0][1].shape)
        print("cls_tars sample_list[0][2]:", sample_list[0][2])
        for i in range(len(sample_list)):
            if i % 10 ==0:
                 display.clear_output(wait=True)
                    
            print("sample index:", i)
            one_sample =  sample_list[i]
#             print("one temp sample:", one_sample)
            # extract feture elements:
            # for paths-->:
            each_dicom_paths = sample_list[i][0]
            
            each_dicom_path0 = each_dicom_paths[0]
            each_dicom_path1 = each_dicom_paths[1]
            each_dicom_path2 = each_dicom_paths[2]
            # for seg_tars-->:
            each_seg_tars = sample_list[i][1]
            
            each_seg_tar0 = each_seg_tars[0]
            each_seg_tar1 = each_seg_tars[1]
            each_seg_tar2 = each_seg_tars[2]
            
            # for cls_tars-->:
            each_cls_tars = sample_list[i][2]
            each_cls_tar0 = each_cls_tars[0]
            each_cls_tar1 = each_cls_tars[1]
            each_cls_tar2 = each_cls_tars[2]
            
            
            # write sample elements into pre-defined proto
            tf_example = image_pair_example(each_dicom_path0, each_dicom_path1, each_dicom_path2, 
                                            each_seg_tar0, each_seg_tar1, each_seg_tar2, 
                                            each_cls_tar0, each_cls_tar1, each_cls_tar2)  # need image_example function to genearte mesage for writing tf records
            writer.write(tf_example.SerializeToString())
#             count_display+=1
#             if count_display >20:
#                 display.clear_output(wait=True)  
    print("Generation of TFRecords is finished.")
        
    
#     for dicom_path, labels in feature_samples_dict.items():
#         if previous_sample is None:
#             previous_sample_sample
#         p =  Path(dicom_path)
# #         path_split = os.path.split(dicom_path)
# #         sub_path_split = os.path.split(path_split[0])
# #         case_name =  sub_path_split[-1]
# #         file_name = path_split[-1] 
# #         file_index =  p.stem()
# #         print("dicom_path:", dicom_path)
# #         print("dicom_path_split:", path_split)
# #         print("sub_path_split:", sub_path_split)
# #         print("case name:", case_name)
# #         print("filename:", file_name)
#         print(p.with_suffix(''))
#         print(p.stem)
#         file_index = int(p.stem)
        
#         print("file index:", file_index)
#         print("file index:", file_index)
        
#     with tf.io.TFRecordWriter(file_nameToSave) as writer:
#         for dicom_path, labels in feature_samples_dict.items():
#             print("dicom_path:", dicom_path)
#     #         dicom_path_string = open(dicom_path, 'rb').read()
#             seg_label = labels[0]
#             print(seg_label.shape)
#             cls_label = labels[1]
# #             tf_example = image_example(dicom_path, seg_label, cls_label)  # need image_example function to genearte mesage for writing tf records
# #             writer.write(tf_example.SerializeToString())
# #             count_display+=1
# #             if count_display >20:
# #                 display.clear_output(wait=True)  
#     print("Generation of TFRecords is finished.")


# def Make_TFRecords(feature_samples_dict, file_nameToSave):
#     # start to write 
#     print("writing tfrecords....")
#     count_display = 0
#     # strat to write postive train dicoms
    
#     with tf.io.TFRecordWriter(file_name) as writer:
#         for dicom_path, labels in feature_samples_dict.items():
#             print("dicom_path:", dicom_path)
#     #         dicom_path_string = open(dicom_path, 'rb').read()
#             seg_label = labels[0]
#             print(seg_label.shape)
#             cls_label = labels[1]
#             tf_example = image_example(dicom_path, seg_label, cls_label)  # need image_example function to genearte mesage for writing tf records
#             writer.write(tf_example.SerializeToString())
#             count_display+=1
#             if count_display >20:
#                 display.clear_output(wait=True)  
#     print("Generation of TFRecords is finished.")
    
    

## Generate Train TFRECORDS

In [3]:
# for train
train_root_folder_path_for_cases= "E:/dataset/Leisang/myTry/BleedingDataDCM/train/"

# give the location of the file
train_excel_loc = "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/TrainfilpOrNot.xlsx"


# make positive and negative samples:

# train_positive_samples, train_negative_samples = make_samples_win(root_folder_path_for_cases=train_root_folder_path_for_cases,
#                                                               excel_loc = train_excel_loc,
#                                                                     train_or_val="train")

# make sample pairs: [(inputs1, seg_tar1, cls_tar1) , （inputs2, seg_tar2, cls_tar2）， (inputs3, seg_tar3, cls_tar3)] 

positive_sample_features_list, negative_sample_features_list =make_sample_pairs(root_folder_path_for_cases=train_root_folder_path_for_cases,
                                                              excel_loc = train_excel_loc,
                                                                    train_or_val="train")


i: 1060
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1061
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1062
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1063
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1064
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1065
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1066
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1067
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1068
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1069
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1070
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1071
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1072
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1073
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1074
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1075
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1076
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 1077
final_

In [4]:
# ## Make into functions for genrerate tf records
# train_positive_file_name =  "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/train_positive_sample_pairs_ubuntu.tfrecords"
# train_negative_file_name =  'E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/train_negative_sample_pairs_ubuntu.tfrecords'
# for lab windows
train_positive_file_name =  "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/train_positive_sample_pairs_WinLab.tfrecords"
train_negative_file_name =  'E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/train_negative_sample_pairs_WinLab.tfrecords'
# make paris dataset for positive
Make_InputPairsTFrecords(positive_sample_features_list,file_nameToSave=train_positive_file_name )

# make paris dataset for negative
Make_InputPairsTFrecords(negative_sample_features_list,file_nameToSave=train_negative_file_name )


sample index: 33510
sample index: 33511
sample index: 33512
sample index: 33513
sample index: 33514
Generation of TFRecords is finished.


## checek the generated TRAIN TFRECORDS dataset

In [ ]:


train_positive_dataset = tf.data.TFRecordDataset(train_positive_file_name)
train_negative_dataset = tf.data.TFRecordDataset(train_negative_file_name)
# Create a dictionary describing the features.
image_feature_description = {
    
        'dicom_path1': tf.io.FixedLenFeature([], tf.string),
        'seg_label1': tf.io.FixedLenFeature([], tf.string), 
        'cls_label1': tf.io.FixedLenFeature([], tf.int64),
        
        'dicom_path2': tf.io.FixedLenFeature([], tf.string),
        'seg_label2': tf.io.FixedLenFeature([], tf.string), 
        'cls_label2': tf.io.FixedLenFeature([], tf.int64),
        
        'dicom_path3': tf.io.FixedLenFeature([], tf.string),
        'seg_label3': tf.io.FixedLenFeature([], tf.string), 
        'cls_label3': tf.io.FixedLenFeature([], tf.int64),
}


def _parse_image_function(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, image_feature_description)
    # Parse the input tf.Example proto using the dictionary above.
    
    # for input 1 --->
    # decode dicom
    dicom_path1 = parsed_features["dicom_path1"]
    image_bytes1 = tf.io.read_file(dicom_path1)
    input_image1 = tf.cast(tfio.image.decode_dicom_image(image_bytes1, dtype=tf.uint16), tf.float32)
    # decode mask
    seg_label1 = tf.cast(tf.io.decode_raw(parsed_features['seg_label1'], tf.uint8), tf.float32)
    seg_label1 = tf.reshape(seg_label1, [-1, 512,512,1])
    
    # for input 2 --->
    # decode dicom
    dicom_path2 = parsed_features["dicom_path2"]
    image_bytes2 = tf.io.read_file(dicom_path2)
    input_image2 = tf.cast(tfio.image.decode_dicom_image(image_bytes2, dtype=tf.uint16), tf.float32)
    # decode mask
    seg_label2 = tf.cast(tf.io.decode_raw(parsed_features['seg_label2'], tf.uint8), tf.float32)
    seg_label2 = tf.reshape(seg_label2, [-1, 512,512,1])
    
    # for input 3 --->
    # decode dicom
    dicom_path3 = parsed_features["dicom_path3"]
    image_bytes3 = tf.io.read_file(dicom_path3)
    input_image3 = tf.cast(tfio.image.decode_dicom_image(image_bytes3, dtype=tf.uint16), tf.float32)
    # decode mask
    seg_label3 = tf.cast(tf.io.decode_raw(parsed_features['seg_label3'], tf.uint8), tf.float32)
    seg_label3 = tf.reshape(seg_label3, [-1, 512,512,1])
    
    return (dicom_path1, dicom_path2, dicom_path3), (input_image1, input_image2, input_image3), (seg_label1, seg_label2, seg_label3),  (parsed_features["cls_label1"], parsed_features["cls_label2"], parsed_features["cls_label3"])


parsed_train_positive_dataset = train_positive_dataset.map(_parse_image_function)
parsed_train_negative_dataset = train_negative_dataset.map(_parse_image_function)

def truncate(x, min, max):
#     print(x.shape)
    cliped =  tf.clip_by_value(x, min, max)
    return cliped
 
def norm(x, min, max):
    # normalize_value = (value − min_value) / (max_value − min_value)
    tensor = tf.math.divide(tf.subtract(x, min),
                    tf.subtract(max, min))
    return tensor

def linear_normalization(input, min=30720.0, max=34816.0):
    truncated_input = truncate(input, min, max)
    norm_input = norm(truncated_input, min, max )
    return  norm_input

def winwise(input,LB,HB):
    # 20 ,380 for range (-32768, 32767)
    # for tf input , (0, 65535)-? LB =  32788, 33148
    input[input<LB] = LB # low boundary , if < LW , set to LW
    input[input>HB] = HB # high boundary, if > Hw, Set to 255
    return input

def plot_one_input(dicom_path, input, seg_label, cls_label):
    palette = copy(plt.cm.gray)
    palette.set_over('r', 1.0)
    print("dicom_path:", dicom_path)
    print("input_image.shape", input.shape)
    print("seg_label", seg_label.shape)
    print("cls_label", cls_label)
    num_nonzeros=np.count_nonzero(seg_label)
    # reshape label 
    target =  tf.reshape(seg_label, input.shape)
    fig, axes = plt.subplots(1,2, figsize=(20,20))
    
    mask =  np.squeeze(target.numpy())
    norm_mask =mask/255.0
# #     input_arr = np.squeeze(winwise(input.numpy(), 32788,33148 ))
    norm_input = linear_normalization(input, 32788.0,33148.0)
    norm_input = np.squeeze(norm_input.numpy())
#     masked_in = norm_mask + norm_input
#     print('masked_in range:[{}, {}]'.format((masked_in.numpy().min()), masked_in.numpy().max()))

#     masked = np.ma.masked_where(norm_mask==0, masked_in)  # this is only generated mask
#     print('masked range:[{}, {}]'.format(masked.min(), masked.max())) 
#     print()
    axes[0].imshow(norm_input, cmap='gray')
    axes[0].set_title('input range:[{}, {}]'.format((norm_input.min()), np.max(norm_input)))
    axes[1].imshow(norm_mask, cmap='gray')
    axes[1].set_title('seg target range:[{}, {}] \n cls_tar: {} # of nonzeros: {}'.format(np.min(norm_mask), np.max(norm_mask), cls_label, num_nonzeros))
    
    print("norm_mask", norm_mask.shape)
    print("norm_input", norm_input.shape)
    masked_in = norm_mask + norm_input
    print('masked_in range:[{}, {}]'.format((masked_in.min()), masked_in.max()))

    masked = np.ma.masked_where(norm_mask==0, masked_in)  # this is only generated mask
    print('masked range:[{}, {}]'.format(masked.min(), masked.max())) 
#     axes[2].imshow(np.squeeze(norm_input), cmap='gray')
#     axes[2].imshow(np.squeeze(masked), palette, colors.Normalize(vmin=0, vmax=1), interpolation='none', alpha=0.4)
#     axes[2].set_title('target range:[{}, {}]'.format(np.min(masked), np.max(masked)) + dicom_path)
    
    fig2, axes2 = plt.subplots(1,1, figsize=(20,20))
    axes2.imshow(norm_input, cmap='gray')
    axes2.imshow(masked, palette, colors.Normalize(vmin=0, vmax=1), interpolation='none', alpha=0.4)
    axes2.set_title('target range:[{}, {}]'.format(np.min(masked), np.max(masked)) + dicom_path)
    
    
    
def check_dataset(dataset):
    # window the input
  

    # print(len(parsed_train_positive_dataset))
    BUFFER_SIZE =512
    # random shuffle the train positive dagtaset
    parsed_dataset = dataset.shuffle(BUFFER_SIZE)
    
 
#     palette.set_under('g', 1.0)
#     palette.set_bad('b', 1.0)
        



    
    for dicom_paths, inputs, seg_labels, cls_labels in parsed_dataset.take(1):
        for i in range(len(dicom_paths)):
            plot_one_input(dicom_paths[i], inputs[i], seg_labels[i], cls_labels[i])
            

       

        
check_dataset(parsed_train_positive_dataset)
check_dataset(parsed_train_negative_dataset)

## Generate Val TFRECORDS

In [5]:
# for val
val_root_folder_path_for_cases= "E:/dataset/Leisang/myTry/BleedingDataDCM/val/"

# give the location of the file
val_excel_loc = "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/valfilpOrNot.xlsx"


# make positive and negative samples:

val_positive_sample_features_list, val_negative_sample_features_list =make_sample_pairs(root_folder_path_for_cases=val_root_folder_path_for_cases,
                                                              excel_loc = val_excel_loc,
                                                                    train_or_val="val")

i: 2100
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2101
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2102
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2103
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2104
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2105
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2106
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2107
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2108
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2109
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
i: 2110
final_cls_list[i]: [0, 0, 0]
num_non_zero_clses 0
total positive samples: 238
total negative samples: 4214


In [6]:
# ## Make into functions for genrerate tf records
# train_positive_file_name =  "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/train_positive_sample_pairs_win.tfrecords"
# train_negative_file_name =  'E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/train_negative_sample_pairs_win.tfrecords'

# val_positive_file_name =  "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/val_positive_sample_pairs_ubuntu.tfrecords"
# val_negative_file_name =  'E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/val_negative_sample_pairs_ubuntu.tfrecords'
# for lab win
val_positive_file_name =  "E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/val_positive_sample_pairs_WinLab.tfrecords"
val_negative_file_name =  'E:\\dataset\\Leisang\\myTry\\BleedingDataDCM/val_negative_sample_pairs_WinLab.tfrecords'
# make paris dataset for positive
Make_InputPairsTFrecords(val_positive_sample_features_list,file_nameToSave=val_positive_file_name )

# make paris dataset for negative
Make_InputPairsTFrecords(val_negative_sample_features_list,file_nameToSave=val_negative_file_name )

sample index: 4210
sample index: 4211
sample index: 4212
sample index: 4213
Generation of TFRecords is finished.


In [ ]:
# make val_positive
# Make_TFRecords(feature_samples_dict=val_negative_samples, file_name=val_negative_file_name)

In [ ]:

val_positive_dataset = tf.data.TFRecordDataset(val_positive_file_name)
val_negative_dataset = tf.data.TFRecordDataset(val_negative_file_name)
parsed_val_positive_dataset = val_positive_dataset.map(_parse_image_function)
parsed_val_negative_dataset = val_negative_dataset.map(_parse_image_function)

check_dataset(parsed_val_positive_dataset)
check_dataset(parsed_val_negative_dataset)